In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from numpy import cov
from numpy.linalg import eig
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve
import scikitplot as skplt

from tqdm import tqdm_notebook

pd.set_option('display.max_columns', 500)
print(os.listdir('/kaggle/input/ieee-fraud-detection'))
print(os.listdir('/kaggle/input/submission-9009'))
# Any results you write to the current directory are saved as output.

In [ ]:
PATH = '/kaggle/input/ieee-fraud-detection'
PATH1 = '/kaggle/input'

In [ ]:
df_train_tran = pd.read_csv(PATH + '/train_transaction.csv')
df_train_tran.head()

In [ ]:
df_train_id = pd.read_csv(PATH + '/train_identity.csv')

In [ ]:
df_train_tran_id = pd.merge(df_train_tran,
                 df_train_id,
                 on='TransactionID', 
                 how='inner')

del df_train_tran, df_train_id
gc.collect()

In [ ]:
print(df_train_tran_id.shape)

# Dropping some columns with more than 50% null values.

In [ ]:
df_train_tran_id['V11'].value_counts(normalize=True)
df_train_tran_id['V1'].isna().sum()/df_train_tran_id.shape[0]

In [ ]:
all_cols = list(df_train_tran_id.columns)
all_null_cols = []
m = df_train_tran_id.shape[0]
for i in range(0,len(all_cols)):
    if all_cols[i] == 'isFraud':
        continue
    percent_na = df_train_tran_id[all_cols[i]].isna().sum()/m
    if percent_na > 0.45:
        print(i)
        print(all_cols[i])
        all_null_cols.append(all_cols[i])
        df_train_tran_id.drop(all_cols[i], axis=1, inplace=True)
#list(df_train_tran.columns)

In [ ]:
df_train_tran_id[all_cols[2]].isna().sum()/m

In [ ]:
len(all_null_cols)

In [ ]:
df_train_tran_id.drop(columns=['TransactionID'], inplace=True)
all_cols = list(df_train_tran_id.columns)
print(len(all_cols))
print(all_cols)

# Cleaning df_train_tran

In [ ]:
df_train_tran_id.head()

In [ ]:
gc.collect()

In [ ]:
cat_cols = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType','DeviceInfo','id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']
cat_cols = list(set(cat_cols).difference(all_null_cols))
n_cat = len(cat_cols)
print(n_cat)

In [ ]:
all_cols = set(df_train_tran_id.columns).difference(set(['TransactionID','isFraud']))
all_cols = list(set(all_cols).difference(all_null_cols))
type(all_cols)
len(all_cols)

In [ ]:
non_cat_cols = list(set(all_cols).difference(set(cat_cols)))
len(non_cat_cols)

Null value imputation for the categorical columns in df_train_tran dataframe

In [ ]:
for col in cat_cols:
    #print(col)
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    if col in list(df_train_tran_id.columns):
        imp.fit(df_train_tran_id[[col]])
        df_train_tran_id[[col]]= imp.transform(df_train_tran_id[[col]])
        df_train_tran_id[[col]] = LabelEncoder().fit_transform(df_train_tran_id[[col]])

In [ ]:
df_train_tran_id.head()

Null value imputation for the non-categorical columns in df_train_tran dataframe

In [ ]:
print(len(non_cat_cols))
for col in non_cat_cols:
    print(col)
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    if col in list(df_train_tran_id.columns):
        print(len(df_train_tran_id[[col]]=='NAN'))
        imp.fit(df_train_tran_id[[col]])
        print('hi')
        df_train_tran_id[[col]]= imp.transform(df_train_tran_id[[col]])
        #df_train_tran_id[[col]] = LabelEncoder().fit_transform(df_train_tran_id[[col]])

In [ ]:
df_train_tran_id.head(10)

# Scaling the data in df_train_tran

In [ ]:
'''df_train_tran_id[np.isfinite(df_train_tran_id) == False] = 0
df_train_tran_id[np.isnan(df_train_tran_id) == True] = 0
df_train_tran_id.fillna()'''

In [ ]:
#df_train_tran_id.dropna(axis=1)
df_train_tran_id = df_train_tran_id.values

In [ ]:
print(type(df_train_tran_id))
print(df_train_tran_id)

In [ ]:
train_tran_x = df_train_tran_id[:,1:]
train_tran_y = df_train_tran_id[:,0]
del df_train_tran_id
gc.collect()
print(np.shape(train_tran_x))
print(np.shape(train_tran_y))

In [ ]:
any(train_tran_y==1)

In [ ]:
for i in range(0,np.shape(train_tran_x)[1]):
    min1 = np.min(train_tran_x[:,i])
    max1 = np.max(train_tran_x[:,i])
    mean = np.mean(train_tran_x[:,i])
    train_tran_x[:,i] = (train_tran_x[:,i] - min1)/(max1-min1)
    #train_tran_x = MinMaxScaler().fit_transform(train_tran_x)

In [ ]:
print(np.shape(train_tran_x))

In [ ]:
train_tran_x[np.isnan(train_tran_x)] = 0
train_tran_x

# Under-sampling the data to remove the bias due to skewness

In [ ]:
rs = RandomUnderSampler(random_state=42)
X, y = rs.fit_resample(train_tran_x, train_tran_y)

In [ ]:
print(X.shape)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score

folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,y), verbose=3, random_state=1001 )

# Here we go
#start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, y)
#timer(start_time) # timing ends here for "start_time" variable

In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

In [ ]:
del X, y
gc.collect()

In [ ]:
df_test_tran = pd.read_csv(PATH + '/test_transaction.csv')
df_test_id = pd.read_csv(PATH + '/test_identity.csv')

In [ ]:
df_test_tran_id = pd.merge(df_test_tran,
                 df_test_id,
                 on='TransactionID', 
                 how='inner')

#del df_test_tran, df_test_id
gc.collect()

In [ ]:
df_test_tran_id.drop(columns=all_null_cols, axis=1, inplace=True)
#df_test_tran_id.drop(columns=['isFraud'], axis=1, inplace=True)
print(np.shape(df_test_tran_id))

In [ ]:
print(np.shape(df_test_tran_id))
#df_test_tran_id.drop(columns=all_null_cols, axis=1, inplace=True)

In [ ]:
df_test_tran_id.head(5)

In [ ]:
for col in cat_cols:
    #print(col)
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    if col in list(df_test_tran_id.columns):
        imp.fit(df_test_tran_id[[col]])
        df_test_tran_id[[col]] = imp.transform(df_test_tran_id[[col]])
        df_test_tran_id[[col]] = LabelEncoder().fit_transform(df_test_tran_id[[col]])
        #df_test_tran_id[[col]] = df_test_tran_id[[col]].astype('category')

In [ ]:
print(len(non_cat_cols))
for col in non_cat_cols:
    #print(col)
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    if col in list(df_test_tran_id.columns):
        imp.fit(df_test_tran_id[[col]])
        df_test_tran_id[[col]]= imp.transform(df_test_tran_id[[col]])
        #df_train_tran[[col]] = LabelEncoder().fit_transform(df_train_tran[[col]])

In [ ]:
list(df_test_tran_id.isnull().any())

In [ ]:
#df_test_tran = df_test_tran.replace(np.nan, 0)

In [ ]:
df_test_tran_id.fillna(0, inplace=True)
#df_test_tran_id.convert_objects(convert_numeric=True)

In [ ]:
df_test_tran_id.head(10)

In [ ]:
df_test_tran_id.head()

In [ ]:
df_test_tran_id = df_test_tran_id.values

In [ ]:
test_tran_x = df_test_tran_id[:,1:]
#del df_test_tran_id
gc.collect()
print(np.shape(test_tran_x))
test_tran_x.astype('int')

In [ ]:
#test_tran_x[np.isnan(test_tran_x)] = 0
#test_tran_x

In [ ]:
for i in range(0,np.shape(test_tran_x)[1]):
    min1 = np.min(test_tran_x[:,i])
    max1 = np.max(test_tran_x[:,i])
    mean = np.mean(test_tran_x[:,i])
    if max1 - min1 != 0:
        test_tran_x[:,i] = (test_tran_x[:,i] - min1)/(max1-min1)

In [ ]:
print(test_tran_x)

In [ ]:
submission = pd.read_csv(PATH + '/sample_submission.csv')
submission_9009 = pd.read_csv(PATH1 + '/submission-9009/ieee_submission_9009.csv')
#submission['isFraud'] = submission_9009['isFraud']
y_test = random_search.predict_proba(test_tran_x)

In [ ]:
#y_test = random_search.predict_proba(test_tran_x)
w1 = 0.6
w2 = 0.4
list_of_tranids_common = list(df_test_tran_id[:,0].astype('int'))
for i in range(0,submission_9009.shape[0]):
    idx = -1
    old_pred = submission_9009.iloc[i,1]
    old_idx = submission_9009.iloc[i,0]
    if old_idx in list_of_tranids_common:
        idx = list_of_tranids_common.index(old_idx)
    if idx >= 0:
        y = y_test[idx,1] # 
        if (y<0.5 and old_pred>=0.5) or (y>=0.5 and old_pred<0.5):
            submission_9009.iloc[i,1] = y
        else:
            submission_9009.iloc[i,1] = (w1 * y) + (w2 * old_pred)
    else:
        submission_9009.iloc[i,1] = old_pred
    
submission_9009.to_csv("submission.csv", index=False)    

In [ ]:
submission_9009.iloc[0,1]

In [ ]:
'''
Plan of Action:-
Assumption: actual values of w1, w2 are same of all data points.
Load the saved params of the model from csv file
Find w1 and w2 based on 2 predictions on train_tran_x and train_tran_id_x with non-linear regression
(with only one sigmoid neuron and log-loss)
Then apply the learned value of w1 and w2 for testing
'''